Import libraries


In [1]:
# import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


Load a raw dataset.


In [2]:
df = pd.read_csv("datasets/dataset_train.csv")


Clean the raw dataset.


In [3]:
def clean_data(dataframe):
    df = dataframe.drop(columns=["Index", "First Name", "Last Name"],
                        errors='ignore')
    return df


df = clean_data(df)

Define describe function.


In [4]:
def describ(df):
    describ_df = pd.DataFrame(
        index=["Count", "Mean", "Std", "Min", "25%", "50%", "75%", "Max"])
    counts = len(df) - df.isna().sum()
    for column in df:
        sorted_df = df.sort_values(column)
        describ_df[column] = [counts[column], 0, 0, 0, 0, 0, 0, 0]
    print(describ_df)


describ(df)


       Hogwarts House  Birthday  Best Hand  Arithmancy  Astronomy  Herbology  \
Count            1600      1600       1600        1566       1568       1567   
Mean                0         0          0           0          0          0   
Std                 0         0          0           0          0          0   
Min                 0         0          0           0          0          0   
25%                 0         0          0           0          0          0   
50%                 0         0          0           0          0          0   
75%                 0         0          0           0          0          0   
Max                 0         0          0           0          0          0   

       Defense Against the Dark Arts  Divination  Muggle Studies  \
Count                           1569        1561            1565   
Mean                               0           0               0   
Std                                0           0               0   
Min    